# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [88]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [89]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,keti bandar,24.1447,67.4497,20.56,82,1,3.88,PK,2023-02-04
1,1,thompson,55.7435,-97.8558,-17.97,88,98,1.25,CA,2023-02-04
2,2,dikson,73.5069,80.5464,-19.98,98,100,1.49,RU,2023-02-04
3,3,busselton,-33.6500,115.3333,22.63,43,75,3.19,AU,2023-02-04
4,4,sidi ali,36.0998,0.4206,8.33,78,0,0.99,DZ,2023-02-04


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [90]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo = True,
    size="Humidity",
    hover_cols=["City"],
    color="City",
    tiles=True,
    alpha=0.4,
    width=900,
    height=500
)

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [91]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Max Temp'] > 18) & 
(city_data_df['Max Temp'] < 26) & (city_data_df['Wind Speed'] < 3) & 
(city_data_df['Cloudiness'] <= 10)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,nazas,25.2333,-104.1333,24.38,20,6,1.35,MX,2023-02-04
20,20,calvinia,-31.4707,19.7760,18.76,57,0,1.89,ZA,2023-02-04
105,105,antofagasta,-23.6500,-70.4000,23.60,72,1,2.84,CL,2023-02-04
110,110,sun city west,33.6620,-112.3413,24.80,20,0,2.24,US,2023-02-04
164,164,mangalagiri,16.4333,80.5500,19.14,89,0,1.06,IN,2023-02-04


### Step 3: Create a new DataFrame called `hotel_df`.

In [92]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,nazas,MX,25.2333,-104.1333,20,
20,calvinia,ZA,-31.4707,19.7760,57,
105,antofagasta,CL,-23.6500,-70.4000,72,
110,sun city west,US,33.6620,-112.3413,20,
164,mangalagiri,IN,16.4333,80.5500,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [93]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
# Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
nazas - nearest hotel: No hotel found
calvinia - nearest hotel: No hotel found
antofagasta - nearest hotel: Hotel Colón
sun city west - nearest hotel: Windmill Suites Surprise, Ascend Hotel Collection
mangalagiri - nearest hotel: No hotel found
otane - nearest hotel: Te Aute Hotel
flinders - nearest hotel: Nova Kiama
viesca - nearest hotel: De los arcos
kyaikkami - nearest hotel: ရှုခင်းသာ
tabou - nearest hotel: hôtel le rochet
sur - nearest hotel: Sur Hotel
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
ulladulla - nearest hotel: Hotel Marlin
benjamin hill - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
clarence town - nearest hotel: Erringhi Hotel
malakal - nearest hotel: No hotel found
port macquarie - nearest hotel: The Observatory
filingue - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
8,nazas,MX,25.2333,-104.1333,20,No hotel found
20,calvinia,ZA,-31.4707,19.7760,57,No hotel found
105,antofagasta,CL,-23.6500,-70.4000,72,Hotel Colón
110,sun city west,US,33.6620,-112.3413,20,"Windmill Suites Surprise, Ascend Hotel Collection"
164,mangalagiri,IN,16.4333,80.5500,89,No hotel found
168,otane,NZ,-39.8833,176.6333,66,Te Aute Hotel
191,flinders,AU,-34.5833,150.8552,57,Nova Kiama
227,viesca,MX,25.3500,-102.8000,18,De los arcos
249,kyaikkami,MM,16.0800,97.5675,76,ရှုခင်းသာ
299,tabou,CI,4.4230,-7.3528,87,hôtel le rochet


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [94]:
%%capture --no-display

hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    color="City",
    size="Humidity",
    hover_cols=["City", "Hotel Name", "Country"],
    tiles=True, 
    alpha=0.3,
    width=900,
    height=500
)
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)